In [1]:
import time
import re
import requests as rt
import bs4
import json
import warnings
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
warnings.filterwarnings("ignore")
import pickle
import numpy
import threading
import numpy as np
import sqlite3 as sq

In [2]:
def unique(list1):
 
    # intilize a null list
    unique_list = []
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    # print list
    for x in unique_list:
        print(x)

In [3]:
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}

In [4]:
def value_tester(index):
    print(product_links[index])
    print(mrp[index])
    print(price[index])
    print(description[index])
    print(gender[index])
    print(brand_names[index])
    print(product_names[index])
    print(primary_image_links_dict[index])
    print(secondary_image_links_dict[index])
    print(sizes[index])

# T-Shirts

In [5]:
product_links={}
c=0
counter=0
while c<3000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&shuffleSearchProducts=true&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscoreBucket17_f%2Bdesc&p_searchquery=tshirt+%7C%7C+tshirt*+%7C%7C+tshirt&s_queryparam=%7B%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5396281%2Bdesc&s_searchquery=tshirt+%7C%7C+tshirt*+%7C%7C+tshirt&isScrapOnlySearch=false&group=false&product_id=&story_id=&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=and&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor=&gender=men&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories=false&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=true&searchFilter=%5B%5B%22classification+.0.1116.1235.1237%22%2C%22classification+.0.11201585.11201684.11201717.11201719%22%2C%22classification+.0.11201585.11201684.11201586.11201595%22%5D%5D&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
        if ('https://www.limeroad.com'+str(i)) not in product_links.values():
            product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
            counter+=1

In [6]:
len(product_links),len(set(product_links.values()))

(2704, 2404)

In [7]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

len(Product_Category)

AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(2600)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1


In [ ]:
len(affiliate_links)

In [ ]:
index=2317
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

In [ ]:
######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass

In [ ]:
con.commit()
con.close()

# Ethnic

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<3000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&shuffleSearchProducts=true&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscoreBucket17_f%2Bdesc&p_searchquery=ethnic+%7C%7C+ethnic*+%7C%7C+ethnic&s_queryparam=%7B%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5398853%2Bdesc&s_searchquery=ethnic+%7C%7C+ethnic*+%7C%7C+ethnic&isScrapOnlySearch=false&group=false&product_id=&story_id=&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=and&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor=&gender=men&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories=false&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=true&searchFilter=%5B%5B%22classification+.0.1116.1240%22%2C%22classification+.0.1116.1240.11202062%22%2C%22classification+.0.11201863.11201864.11201896.11201907%22%2C%22classification+.0.11201585.11201684.11201639.11201709%22%5D%5D&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
        if ('https://www.limeroad.com'+str(i)) not in product_links.values():
            product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
            #print(i.get('href'))
            counter+=1

In [ ]:
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

len(Product_Category)

In [ ]:
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(2300)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1

In [ ]:
len(affiliate_links)

In [ ]:
index=2700
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

In [ ]:
######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass

In [ ]:
con.commit()
con.close()

# Tops Women


In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<3000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=2&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1235.1236%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1235.1236%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5398879%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1236st&s_rows=2&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
        if ('https://www.limeroad.com'+str(i)) not in product_links.values():
            product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
            #print(i.get('href'))
            counter+=1


In [ ]:
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

len(Product_Category)

In [ ]:
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(2000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1

In [ ]:
len(affiliate_links)

In [ ]:
index=2703
product_links[index],affiliate_links[index]

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

In [ ]:
######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass

In [ ]:
con.commit()
con.close()

# Tees Women

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<3000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=2&story_start=15&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1235.1237%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1235.1237%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5398910%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1237st&s_rows=2&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
        if ('https://www.limeroad.com'+str(i)) not in product_links.values():
            product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
            #print(i.get('href'))
            counter+=1


In [ ]:
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

len(Product_Category)

In [ ]:
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(2680)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1

In [ ]:
len(affiliate_links)

In [ ]:
index=2657
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

In [ ]:
######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass

In [ ]:
con.commit()
con.close()

# Shirts Women

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<3000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=1&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1235.11201443%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1235.11201443%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5398923%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201443st&s_rows=2&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
        if ('https://www.limeroad.com'+str(i)) not in product_links.values():
            product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
            #print(i.get('href'))
            counter+=1


In [ ]:
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

len(Product_Category)

In [ ]:
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(2200)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1

In [ ]:
len(affiliate_links)

In [ ]:
index=2317
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

In [ ]:
######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass

In [ ]:
con.commit()
con.close()

# Women Tunics

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<3000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=4&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1235.11201447%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1235.11201447%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399139%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201447st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
        if ('https://www.limeroad.com'+str(i)) not in product_links.values():
            product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
            #print(i.get('href'))
            counter+=1


In [ ]:
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

len(Product_Category)

In [ ]:
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(2000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1

In [ ]:
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Dresses

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<3000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=2&story_start=4&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1235.1238%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1235.1238%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399142%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1238st&s_rows=2&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
        if ('https://www.limeroad.com'+str(i)) not in product_links.values():
            product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
            #print(i.get('href'))
            counter+=1


In [ ]:
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

len(Product_Category)

In [ ]:
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(2000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1

In [ ]:
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Jumpsuits

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<2000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1235.1239%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1235.1239%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399148%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1239st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1


In [ ]:
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

len(Product_Category)

In [ ]:
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(2000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1

In [ ]:
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Capes and Shrugs

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<1000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1235.11201455%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1235.11201455%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399150%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201455st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1


In [ ]:
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

len(Product_Category)

In [ ]:
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(300)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1

In [ ]:
len(affiliate_links)

In [ ]:
index=300
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Jeans/Leggings

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<2500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=2&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1217.1248%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1217.1248%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399153%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1248st&s_rows=2&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(200)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=200
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Skirts

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<1500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=2&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1217.1247.1256%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1217.1247.1256%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399154%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1256st&s_rows=2&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

Print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Plazzos 

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<2000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=1&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1217.11201459%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1217.11201459%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399155%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201459st&s_rows=2&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Shorts

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<1000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1217.1247.1257%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1217.1247.1257%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399157%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1257st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))

description=product_names
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Capris


In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<1000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1217.1246.1252%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1217.1246.1252%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399160%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1252st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
description=product_names
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Trousers

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<2000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1217.1249%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1217.1249%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399161%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1249st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=100
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Track Pants

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<1000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1217.11201460%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1217.11201460%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399162%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201460st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Bras

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<3000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscoreBucket17_f%2Bdesc&p_searchquery=bras+%7C%7C+bras*+%7C%7C+bra&s_queryparam=%7B%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399163%2Bdesc&s_searchquery=bras+%7C%7C+bras*+%7C%7C+bra&isScrapOnlySearch=false&group=false&product_id=&story_id=&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=and&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor=&gender=men&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories=false&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=true&searchFilter=%5B%5B%22classification+.0.1116.1128.1129%22%2C%22classification+.0.1116.1128.1226%22%2C%22classification+.0.1116.1128.1129%22%5D%5D&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Panties


In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<2000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscoreBucket17_f%2Bdesc&p_searchquery=panties+%7C%7C+panties*+%7C%7C+panty&s_queryparam=%7B%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399170%2Bdesc&s_searchquery=panties+%7C%7C+panties*+%7C%7C+panty&isScrapOnlySearch=false&group=false&product_id=&story_id=&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=and&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor=&gender=men&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories=false&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=true&searchFilter=%5B%5B%22classification+.0.1116.1128.1227%22%5D%5D&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Night Wear


In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<3000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscoreBucket17_f%2Bdesc&p_searchquery=night+wear+%7C%7C+night+wear*+%7C%7C+night+wear&s_queryparam=%7B%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399172%2Bdesc&s_searchquery=night+wear+%7C%7C+night+wear*+%7C%7C+night+wear&isScrapOnlySearch=false&group=false&product_id=&story_id=&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=and&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor=&gender=men&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories=false&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=true&searchFilter=%5B%5B%22classification+.0.1116.1128.11201469%22%2C%22classification+.0.11201585.11201684.11201598.11201603%22%2C%22classification+.0.1116.1128.11201469%22%2C%22classification+.0.11201585.11201684.11201598.11201603%22%5D%5D&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=100
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women sweatshirts

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<2500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.11201399%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.11201399%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399174%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201399st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Jackets

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<1500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.1232.1260%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.1232.1260%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399175%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1260st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

#  Women Blazer and Coats

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<800:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=1&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.1232.1261%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.1232.1261%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399176%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1261st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Pullover

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<800:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.1234%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.1234%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399178%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1234st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(500)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=500
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Woolen Kurta

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<200:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.11201484%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.11201484%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399179%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201484st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Stoles

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<1500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.11201483%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.11201483%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399180%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201483st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Shawls

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<1000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.1233%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.1233%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399181%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1233st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Poncho

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<100:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.11201490%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.11201490%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399182%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201490st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
description=product_names
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(20)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=20
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Mufflers, Caps, Gloves

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<100:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.11201485%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.11201485%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399183%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201485st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(35)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=35
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Thermal Innerwear

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<100:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.11201787%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1116.1230.11201787%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399184%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201787st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(30)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=30
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Sandals

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<2000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1148.1150.1153%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1148.1150.1153%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399232%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1153st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(1000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=1000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Ballerina

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<1000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1148.1150.1151%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1148.1150.1151%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399234%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1151st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(1000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=1000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Casual Shoes

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1148.11201442%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1148.11201442%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399236%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201442st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[1].text.strip()[2:] +", "+ cat[2].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(1000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=1000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Sports Shoes

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=1&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1148.11201512.11201691%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1148.11201512.11201691%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399240%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201691st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=1000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Sneakers Women

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=1&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1148.11201512.11201692%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1148.11201512.11201692%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399242%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201692st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=1000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Loafers & Mocassins

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1148.1152%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1148.1152%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399243%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1152st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=1000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Flip-Flop

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1148.1158%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1148.1158%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399244%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1158st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[1].text.strip()[2:] +", "+ cat[2].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=1000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Boots

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1148.1149%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1148.1149%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399245%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1149st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[1].text.strip()[2:] +", "+ cat[2].text.strip()[2:]

print(len(Product_Category))
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=100
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Pumps

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1148.1154.1156%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1148.1154.1156%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399287%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms1156st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=100
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Women Wedges

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.1148.1154.11201043%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.1148.1154.11201043%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399288%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201043st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=100
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men t-shirts

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<3000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=2&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201595%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201595%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399289%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201595st&s_rows=2&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
                if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                    product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                    print(i.get('href'))
                    counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=100
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Tshirt Polo

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<2000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22tag_values%22%3A%7B%22type%22%3A%5B%22polo+t-shirt%22%5D%2C%22neck%22%3A%5B%22polo+neck%22%5D%7D%2C%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201595%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22tag_values%22%3A%7B%22type%22%3A%5B%22polo+t-shirt%22%5D%2C%22neck%22%3A%5B%22polo+neck%22%5D%7D%2C%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201595%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399291%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201595st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Casual Shirts

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<3000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11202215.11201587%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11202215.11201587%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399291%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201587st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=100
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Formal Shirts

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<4000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=1&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11202215.11201588%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11202215.11201588%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399292%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201588st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Jeans

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<2500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=2&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201632%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201632%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399292%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201632st&s_rows=2&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Casual Trouser

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<2000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201598.11201599%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201598.11201599%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399293%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201599st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict)

In [ ]:
value_tester(1001)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=1001
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Formal Trouser

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<1000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201598.11201600%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201598.11201600%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399293%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201600st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=100
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# 

# Joggers

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<1500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201684.11201598.11201861%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscoreBucket6_f%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201684.11201598.11201861%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399294%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201861st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
description=product_names   
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Shorts

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<1500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=4&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201684.11201598.11201601%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscoreBucket6_f%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201684.11201598.11201601%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399294%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201601st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Kurtas

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<2000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201592%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201592%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399295%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201592st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=100
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Ethnic Wear Sets

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<3000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201695%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201695%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399296%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201695st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Nehru Jackets

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<1500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22categories%22%3A%5B%22Men%2FClothing%2Ftop+wear%2FJackets%2FNehru+Jacket%22%5D%2C%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201593%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscoreBucket6_f%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22categories%22%3A%5B%22Men%2FClothing%2Ftop+wear%2FJackets%2FNehru+Jacket%22%5D%2C%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201593%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399297%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201593st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Briefs

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<1500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201608.11201610%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201608.11201610%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399299%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201610st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
value_tester(1000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=1000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Boxers

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<1000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201608.11201792%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201608.11201792%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399300%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201792st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Vests

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201608.11201609%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201608.11201609%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399300%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201609st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
values_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Sneakers

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201685.11201612.11201693%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscoreBucket6_f%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201685.11201612.11201693%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399301%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=14945084&story_id=ms11201693st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=false&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&src_id=5afe78ece667f__43&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    Product_Category[i] = s2.findAll("a", {"class": "tdN c6 hcP"})cat[2].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
values_tester(1000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Mens Sports Shoes

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<1000:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201685.11201614%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscoreBucket6_f%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201685.11201614%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399305%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=14147693&story_id=ms11201614st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=false&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&src_id=5afe78ece67d3__43&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    Product_Category[i] = s2.findAll("a", {"class": "tdN c6 hcP"})cat[2].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
values_tester(1000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Sandals Floaters

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<800:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201685.11201714%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscoreBucket6_f%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201685.11201714%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399307%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=14933072&story_id=ms11201714st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=false&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&src_id=5afe78ece6922__43&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    Product_Category[i] = s2.findAll("a", {"class": "tdN c6 hcP"})cat[2].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
value_tester(400)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=400
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Boots

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201685.11201612.11201694%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscoreBucket6_f%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201685.11201612.11201694%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399319%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=13304869&story_id=ms11201694st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=false&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&src_id=5afe78ece6ae5__43&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    Product_Category[i] = s2.findAll("a", {"class": "tdN c6 hcP"})cat[2].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
values_tester(1000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Slip-ons

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201685.11201612.11201843%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscoreBucket6_f%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201685.11201612.11201843%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399321%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=14127733&story_id=ms11201843st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=false&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&src_id=5afe78ece6d07__43&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    Product_Category[i] = s2.findAll("a", {"class": "tdN c6 hcP"})cat[2].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=100
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Loafers

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<900:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201685.11201612.11201636%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscoreBucket6_f%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201685.11201612.11201636%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399322%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=12901876&story_id=ms11201636st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=false&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&src_id=5afe78ece71f8__43&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    Product_Category[i] = s2.findAll("a", {"class": "tdN c6 hcP"})cat[2].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
values_tester(1000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Casual jacket

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<900:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201593.11201743%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201593.11201743%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399323%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201743st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
values_tester(1000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Biker jackets

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201593.11201738%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201593.11201738%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399326%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201738st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))

description=product_names
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=100
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# MEn Quilted Jackets

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201593.11201802%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201593.11201802%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399326%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201802st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
values_tester(1000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Bomber Jacket

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201593.11201801%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201593.11201801%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399327%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201801st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
values_tester(1000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Denim Jacket

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=1&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201593.11201741%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201593.11201741%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399327%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201741st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=100
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Suits

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201594%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201594%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399328%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201594st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
values_tester(1000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Blazers

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201726%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201726%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399328%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201726st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
values_tester(1000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Sweatshirts

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<500:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=2&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201597%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201597%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399329%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201597st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
value_tester(100)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=100
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Pullover

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<700:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=0&story_start=0&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201596.11201702%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscoreBucket6_f%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22stock%22%3A%5B%221%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201684.11201586.11201596.11201702%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399330%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201702st&s_rows=0&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=0&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
values_tester(1000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    

# Men Shrugs

In [ ]:
###  Product LINks
product_links={}
c=0
counter=0
while c<700:
    data=rt.request('get',f"https://www.limeroad.com/listing/get_listing_objects?p_start={c}&s_start=2&story_start=4&stories_to_edit=false&scraps_to_edit=false&stories_to_rate=false&scraps_to_rate=false&p_queryparam=%7B%22categories%22%3A%5B%22Men%2FClothing%2Ftop+wear%2Fshrugs%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201684.11201586%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&p_sortorder=threeQuarterStock_i%2Bdesc%2CscCount%2Bdesc&p_searchquery=*%3A*&s_queryparam=%7B%22categories%22%3A%5B%22Men%2FClothing%2Ftop+wear%2Fshrugs%22%5D%2C%22classification%22%3A%5B%22.0.11201585.11201684.11201586%22%5D%2C%22stock%22%3A%5B%221%22%5D%7D&s_sortorder=priority%2Bdesc%2Crandom_5399331%2Bdesc&s_searchquery=*%3A*&isScrapOnlySearch&group=false&product_id=&story_id=ms11201586st&s_rows=2&p_rows=8&tag_sort=&df_type=&tag_condition=%22%22&tag=&p_groupoffset=&story_rows=2&stories&ext_img=&template=&badge&catEditor&gender&chopPage&chopPageSameUser&filterUserid&disableFixedScraps&tagPageStories&newUserSort=bsSort&new_user_data=newUserFlow&st_queryparam=&st_queryparam_or=&issearch=false&ajax=true",headers=headers)
    c+=8
    data=bs4.BeautifulSoup(data.text,'html.parser')
    for i in data.find_all('a',{'data-obj':"impVipAjaxMenuCloseLog"}):
            if ('https://www.limeroad.com'+str(i)) not in product_links.values():
                product_links[counter]='https://www.limeroad.com'+str(i.get('href'))
                #print(i.get('href'))
                counter+=1
len(product_links),len(set(product_links.values()))

In [ ]:
###################### product details    1.1 diff primary and secondary img
outstock=0
brand_names={}
product_names={}
primary_image_links_dict={}
secondary_image_links_dict={}
sizes={}
price={}
mrp={}
description={}
gender={}
Product_Category={}
for i in range(len(product_links)):
    data2=rt.request('get',product_links[i])
    s2=bs4.BeautifulSoup(data2.text,'html.parser')
    if s2.find('div',{'class':'p40 pt0'}):
        if s2.find('div',{'class':'p40 pt0'}).text == 'This product is out of stock!':
            outstock+=1
            continue
    brand_names[i]=s2.find('div',{'class':'fs12 p12 bs pt0 pb0 taL dT wp100'}).find('a').text #fs12 p12 bs pt0 pb0 taL dT wp100
    product_names[i]=s2.find('h1',{'class':'ftwN m0 p0 c3 fs20 pt0 p12 pb2 bs ttC'}).text
    price[i]=int(s2.find('span',{'class':'sell'}).text)
    try:
        mrp[i]=int(s2.find('span',{'class':'mrp'}).text)
    except:
        mrp[i]=int(s2.find('span',{'class':'sell'}).text)
    gender[i]=s2.find_all('span',{'itemprop':'name'})[1].text
    #description[i]=s2.find_all('div',{'class':'p80 pt0 wp100'})[1].find('div',{'class':'dIb vT c3'}).text
    for m in s2.find_all('div',{'class':'p80 pt0 wp100'}):
        if re.match('\nproduct details :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
        if re.match('\nproduct description :',m.text,re.IGNORECASE):
            description[i]=m.find('div',{'class':'dIb vT c3'}).text
    temp_primary_img=s2.find('div',{'class':'m02 vT dIb pR zmimgH w310 h412 z2'})
    primary_image_links_dict[i]=temp_primary_img.find('img').get('data-src')[2:]
    image_links=[]
    for k in s2.find_all('div',{'class':'m02 zmimgH vT dIb pR w310 h412 z2'}):
        image_links.append(k.find('img').get('data-src')[2:])
    image_links=image_links[1:]
    secondary_image_links_dict[i]=[j for j in image_links if j]
    temp=s2.find('div',{'id':'alSz'})
    temp_sizes=[]
    for j in temp.find_all('div',{'class':'br4 dIb m4 vT p12 fs16 bxs bs ttU taC wsN size bd3 hcP hbxs'}):
        if int(j.attrs['data-e']) > 0:
            temp_sizes.append(j.attrs['data-b'][5:])
    sizes[i]=temp_sizes
    cat = s2.findAll("a", {"class": "tdN c6 hcP"})
    Product_Category[i] = cat[2].text.strip()[2:] +", "+ cat[3].text.strip()[2:]

print(len(Product_Category))
    
len(product_links),len(mrp),len(price),len(description),len(gender),len(brand_names),len(product_names),len(primary_image_links_dict),len(secondary_image_links_dict),len(sizes)

In [ ]:
values_tester(1000)

In [ ]:
#To get affiliate links
affiliate_links={}
br=webdriver.Chrome()
br.get("https://www.cuelinks.com/login")
time.sleep(2)
email=br.find_element_by_id('user_email')
password=br.find_element_by_id('user_password')
email.send_keys('sm@grroom.in')
password.send_keys('cuelinks@grroom333')
time.sleep(2)
button=br.find_element_by_class_name('waves-button-input')
button.send_keys(Keys.ENTER)
br.get("https://www.cuelinks.com/link-kit")
time.sleep(2)
counter=0

for i in range(len(product_links)):   
    url=br.find_element_by_class_name('validate')
    url.send_keys(product_links[i])
    button2=br.find_element_by_tag_name('button')
    button2.send_keys(Keys.ENTER)
    time.sleep(1)
    text_area=br.find_element_by_name('link[affiliatized_url]')
    affiliate_links[i]=text_area.get_attribute('value')
    text_area.clear()
    url.clear()
    text_area.send_keys(Keys.CONTROL, 'a',Keys.DELETE)
    counter+=1
    
len(affiliate_links)

In [ ]:
index=2000
print(product_links[index])
print(affiliate_links[index])

In [ ]:
###################################################                 1.1
con=sq.connect(r"E:\GRROOM\DataBase\LimeRoad2.db")  
cur=con.cursor()

######################################################              1.1
for i in range(len(product_links)):
    try:
        if i in Product_Category.keys() and i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links_dict.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys() and i in affiliate_links.keys() and i in primary_image_links_dict.keys():
            if Product_Category[i]!= ' ' and brand_names[i] != '' and primary_image_links_dict[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links_dict[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' and affiliate_links[i] != '':
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)',
                            ('https://www.limeroad.com/',
                             product_links[i],
                             product_names[i],
                             brand_names[i],
                             Product_Category[i]
                             ','.join(sizes[i]),
                             price[i],
                             mrp[i],
                             gender[i],
                             description[i],
                             primary_image_links_dict[i],
                             ','.join(secondary_image_links_dict[i]),
                             affiliate_links[i])
                           )
    except:
        pass
    
con.commit()
con.close()    